In [1]:
import os
import io 
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as sgt 
import statsmodels.tsa.stattools as sts 
from statsmodels.tsa.seasonal import seasonal_decompose 
from scipy.stats.distributions import chi2
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [33]:
output_dir = 'data'
os.makedirs(output_dir, exist_ok = True)

input_dir = 'data'
file = 'mock_data.csv'

output_file = 'output_file.xlsb'
output_file_path = os.path.join(output_dir, output_file)


output_excel_dir = 'results_'
os.makedirs(output_excel_dir, exist_ok = True)

In [3]:
# Read the CSV file using pandas
csv_file_path = os.path.join(input_dir, file)
df = pd.read_csv(csv_file_path)

In [4]:
# Function to extract the year from a date
def extract_year(date_str):
    # Split the date string by '-' and get the first element (the year)
    return date_str.split('-')[0]

# Apply the function to create a new 'Year' column
df['Year'] = df['date'].apply(lambda x: extract_year(x))
df['Year'] = df['Year'].astype(int)


In [5]:
#sort the date
df = df.sort_values(by = ['team_request_type', 'date'])                          

In [7]:
df['date'] = pd.to_datetime(df['date'])
# merged_data.index.freq = 'm'
# merged_data.set_index("date", inplace=true)
# convert date strings to datetime objects

train_start_date = pd.to_datetime('2020-01-01')
train_end_date = pd.to_datetime('2023-05-01')

valid_start_date = pd.to_datetime('2023-06-01')
valid_end_date = pd.to_datetime('2023-08-01')


train = df[(df['date'] >= train_start_date) & (df['date'] <= train_end_date)]
valid = df[(df['date'] >= valid_start_date) & (df['date'] <= valid_end_date)]



In [8]:
col_to_exclude = ['Year']
train = train.drop(columns = col_to_exclude)
valid = valid.drop(columns = col_to_exclude)

df.rename(columns = {'date':'ds' , 'team_request_type':'unique_id' , 'volume':'y'}, inplace = True) 

train.rename(columns = {'date':'ds' , 'team_request_type':'unique_id' , 'volume':'y'}, inplace = True) 
valid.rename(columns = {'date':'ds' , 'team_request_type':'unique_id' , 'volume':'y'}, inplace = True) 


# Holtwinters Model

In [ ]:
mae_df = pd.DataFrame(columns = ['unique_id' , 'MAE'])

unique_ids = train['unique_id'].unique()
result_dict = {}

for target_unique_id in unique_ids:
    train_subset = train[train['unique_id']==target_unique_id]
    valid_subset = valid[valid['unique_id']==target_unique_id]
    
    train_subset['ds'] = pd.to_datetime(train_subset['ds'])
    train_subset.set_index(pd.DatetimeIndex(train_subset['ds'], freq = 'MS'),inplace = True)
    
    valid_subset['ds'] = pd.to_datetime(valid_subset['ds'])
    valid_subset.set_index(pd.DatetimeIndex(valid_subset['ds'], freq = 'MS'),inplace = True)
    
    
#     df['ds'] = pd.to_datetime(df['ds'])
#     df.set_index(pd.DatetimeIndex(df['ds'], freq = 'MS'),inplace = True)
    
    
    # Capping in 95th & 5th percentile value
    
    p95 = np.percentile(train_subset['y'],95)
    p5 = np.percentile(train_subset['y'],5)
    
    train_subset['y'] = np.where(train_subset['y'] > p95,p95, train_subset['y'])
    train_subset['y'] = np.where(train_subset['y'] > p5,p5, train_subset['y'])
    
    # define hyper parameters 
    alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    betas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    gammas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    seasonal_periods = [12]
    
    best_mae = float('inf')
    best_params = None
    
    for trend_type in ['add','mul']:
        for alpha in alphas:
            for beta in betas:
                for gamma in gammas:
                    for seasonal_period in seasonal_periods:
                        model = sm.tsa.ExponentialSmoothing(train_subset['y'], trend = trend_type, seasonal = 'add' , seasonal_periods = seasonal_period)
                        model_fit = model.fit(smoothing_level = alpha, smoothing_slope = beta, smoothing_seasonal = gamma , optimized = False)
                        
                        forecast = model_fit.forecast(len(valid_subset))
                        
                        mae = mean_absolute_error(valid_subset['y'], forecast)
                        
                        if mae < best_mae:
                            best_mae = mae
                            best_params =  (trend_type, alpha, beta, gamma, seasonal_period)
                            
                        mae_df = mae_df.append({'unique_id':target_unique_id, 'MAE': best_mae},ignore_index = True)
                        
    best_trend, best_alpha, best_beta, best_gamma, best_seasonal_period = best_params 
    
    # refit the best model with best hyper parameters
    best_model = sm.tsa.ExponentialSmoothing(train_subset['y'], trend=  best_trend, seasonal=  'add', seasonal_periods=best_seasonal_period)
    best_model_fit = best_model.fit(smoothing_level = best_alpha, smoothing_slope = best_beta, smoothing_seasonal = best_gamma )

    future_forecasts = best_model_fit.forecast(steps = 6) # forecasting months
    
    forecast_dates = pd.date_range(start = valid_subset.index[-1], periods = 6, freq = 'MS')
    future_forecast_df = pd.DataFrame({'ds': forecast_dates, 'forecasted_value': future_forecasts})
    
    #plot the charts
    plt.figure(figsize = (12,6))
    plt.plot (train_subset.index, train_subset['y'], label = 'Training data' , linestyle = '--', marker = 'o')
    plt.plot(future_forecast_df['ds'], future_forecast_df['forecasted_value'], label='forecast_volume', linestyle='--', marker='o', color='green')
    plt.xlabel('date')
    plt.ylabel('volume')
    plt.legend()
    plt.title(f"unique ID: {target_unique_id} - {best_trend} Time Series Forecast Comparison")
    
    print(f" best model mae for {target_unique_id} : {best_mae}")
    
    
    # Create a dictionary to store actual vs. predicted values for each unique ID
    result_dict = {}

# Loop through unique IDs
    for target_unique_id in unique_ids:
        # Extract actual and predicted values for the current unique ID
        actual_values = valid_subset['y'].tolist()
        predicted_values = future_forecasts.tolist()

        # Find the maximum length of actual and predicted values
        max_len = max(len(actual_values), len(predicted_values))

        data_list = []

        # Iterate through the data points
        for i in range(max_len):
            actual_value = actual_values[i] if i < len(actual_values) else None
            predicted_value = predicted_values[i] if i < len(predicted_values) else None

            # Create a dictionary for the data point
            data_list.append({
                'ds': i + 1,
                'unique_id': target_unique_id,
                'actual_values': actual_value,
                'predicted_values': predicted_value
            })

        # Create a DataFrame from the data list
        df = pd.DataFrame(data_list)

        # Define the output Excel file path
        output_excel_file = os.path.join(output_excel_dir, f'{target_unique_id}_forecast_results.xlsx')

        # Save the DataFrame to an Excel file
        df.to_excel(output_excel_file, index=False)

        # Store the actual vs. predicted values in the result dictionary
        result_dict[target_unique_id] = {
            'actual_values': actual_values,
            'predicted_values': predicted_values
        }

 best model mae for Team A - Request 1 : 87.52582637770672
 best model mae for Team A - Request 10 : 194.31770855882863
 best model mae for Team A - Request 11 : 600.9412077947063
 best model mae for Team A - Request 12 : 74.65120275673219
 best model mae for Team A - Request 13 : 250.0985676207357
 best model mae for Team A - Request 14 : 405.35705096448834
 best model mae for Team A - Request 15 : 483.09841370769954
 best model mae for Team A - Request 16 : 472.68055344571576
 best model mae for Team A - Request 17 : 381.52689738687104
 best model mae for Team A - Request 18 : 504.0960474980002
 best model mae for Team A - Request 19 : 122.17912430772769
 best model mae for Team A - Request 2 : 599.6155374912984
 best model mae for Team A - Request 20 : 200.1102496666667
 best model mae for Team A - Request 3 : 206.58344018310459
 best model mae for Team A - Request 4 : 322.4664834079936


/opt/conda/envs/anaconda-2022.05-py39/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


 best model mae for Team A - Request 5 : 138.1504185145815
 best model mae for Team A - Request 6 : 411.6989488428112
 best model mae for Team A - Request 7 : 670.7749121098907
 best model mae for Team A - Request 8 : 149.29258529098396
 best model mae for Team A - Request 9 : 271.54318225541994


/opt/conda/envs/anaconda-2022.05-py39/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


 best model mae for Team B - Request 1 : 146.19390042542656
 best model mae for Team B - Request 10 : 822.0070023989991
 best model mae for Team B - Request 11 : 567.1779043602642
 best model mae for Team B - Request 12 : 235.8164093478881
 best model mae for Team B - Request 13 : 320.66687696419655
 best model mae for Team B - Request 14 : 307.032468025568
 best model mae for Team B - Request 15 : 356.4658919138288
 best model mae for Team B - Request 16 : 108.21014252591199
 best model mae for Team B - Request 17 : 262.84926375658654
 best model mae for Team B - Request 18 : 484.88427646640986
